In [0]:
%sh 
curl -O https://an-big-data-assignment-2.s3.us-east-2.amazonaws.com/COVID-19.zip
unzip COVID-19.zip &>/dev/null

% Total % Received % Xferd Average Speed Time Time Time Current
 Dload Upload Total Spent Left Speed

 0 0 0 0 0 0 0 0 --:--:-- --:--:-- --:--:-- 0
 0 0 0 0 0 0 0 0 --:--:-- --:--:-- --:--:-- 0
 9 157M 9 15.1M 0 0 12.1M 0 0:00:12 0:00:01 0:00:11 12.1M
 28 157M 28 45.4M 0 0 20.2M 0 0:00:07 0:00:02 0:00:05 20.2M
 48 157M 48 76.7M 0 0 23.5M 0 0:00:06 0:00:03 0:00:03 23.5M
 68 157M 68 107M 0 0 25.3M 0 0:00:06 0:00:04 0:00:02 25.3M
 88 157M 88 138M 0 0 26.4M 0 0:00:05 0:00:05 --:--:-- 27.9M
100 157M 100 157M 0 0 26.9M 0 0:00:05 0:00:05 --:--:-- 30.9M

In [0]:
display(dbutils.fs.ls('file:/databricks/driver/COVID-19/'))

path,name,size
file:/databricks/driver/COVID-19/Normal/,Normal/,4096
file:/databricks/driver/COVID-19/Viral_Pneumonia/,Viral_Pneumonia/,4096
file:/databricks/driver/COVID-19/Covid/,Covid/,4096


In [0]:
Normal = "/tmp/Normal"
Covid = "/tmp/Covid"
Viral_Pneumonia = "/tmp/Viral_Pneumonia"

In [0]:
dbutils.fs.cp('file:/databricks/driver/COVID-19/Normal/', '/tmp/Normal', recurse=True)
dbutils.fs.cp('file:/databricks/driver/COVID-19/Covid/', '/tmp/Covid', recurse=True)
dbutils.fs.cp('file:/databricks/driver/COVID-19/Viral_Pneumonia/', '/tmp/Viral_Pneumonia', recurse=True)

Out[5]: True

In [0]:
display(dbutils.fs.ls('tmp/'))

path,name,size
dbfs:/tmp/2e8cfe203a6f_cdx/,2e8cfe203a6f_cdx/,0
dbfs:/tmp/Covid/,Covid/,0
dbfs:/tmp/Normal/,Normal/,0
dbfs:/tmp/Viral Pneumonia/,Viral Pneumonia/,0
dbfs:/tmp/Viral_Pneumonia/,Viral_Pneumonia/,0
dbfs:/tmp/circles/,circles/,0
dbfs:/tmp/hive/,hive/,0
dbfs:/tmp/squares/,squares/,0
dbfs:/tmp/triangles/,triangles/,0


In [0]:
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml import Pipeline
from sparkdl import DeepImageFeaturizer
from pyspark.sql.functions import lit
from sparkdl.image import imageIO
img_dir = '/tmp'

/databricks/python/lib/python3.7/site-packages/botocore/vendored/requests/packages/urllib3/_collections.py:1: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
 from collections import Mapping, MutableMapping
Using TensorFlow backend.

In [0]:
Normal_df = spark.read.format("image").load(img_dir + "/Normal").withColumn("label", lit(0))
Covid_df = spark.read.format("image").load(img_dir + "/Covid").withColumn("label", lit(1))
Viral_Pneumonia_df = spark.read.format("image").load(img_dir + "/Viral_Pneumonia").withColumn("label", lit(2))

In [0]:
display(Viral_Pneumonia_df)

image label List(dbfs:/tmp/Viral_Pneumonia/01.jpeg, 1333, 1582, 3, 16, Binary image data placeholder. Access the image data field directly to view raw binary data., iVBORw0KGgoAAAANSUhEUgAAAGMAAABUCAIAAAAZJh7XAAAs3UlEQVR42o3cefxuY7nH8Z1hb/Y2ZetQylwSMqcoSYPCaSASMjRKqFBJadAgaZAGpdkp6iSnDM1lDJGpKJRhyxDHlIxNr/N+1mfvtYejc7r/WL/1W8967uF7fa/vdd33utczZcq/UB4xT1loTll4KIssssjUqVMXW2yxxRdffPr06UssscQjH/nIFVZYYeWVV37CE56w0UYbPf3pT3/+85//kpe85GUve9luu+326le/+i1vecsRRxzxpS996Tvf+c5JJ530kY985MADDzzggAPe9ra3fWwoH/rQhw477LAjjzzyE5/4hOP++++/xx577LnnnrvuuutOO+2kti233HKzzTbbeOON11577dVXX/2xj33svw1l+eWXd1xuueVmzpy57LLLLrXUUro0bdo0PVx00UUXmb8sOk9xjyFMG8rUOaWPuvlfgmYBgObFqDamD2XGjBlgWnLJJSH1qEc96nGPe9xaa6311Kc+9dnPfvYLX/hCSO28885veMMbIPKe97wHUp/+9Kf/YyhO3vGOd7zvfe8D2XHHHfeNb3zjK1/5CoxA9uEPfxhkPgXu6173OijDWlUveMELnvOc56h5/fXX1wqkYKRRFgqv5YYCr8AKhan/q7i+xPxlxlAWH8q833oYUACxAC7/DKNaCiMFRrq19NJL6+tjHvOYVVdd9clPfjKktttuO0RAh1e96lX77befMb/97W+PU0D5r6HAxZWPfvSjgPvOUL75zW+eeOKJxx9//Cc/+cn3vve9b3zjG1//+te/5jWv2XvvvXffffeXvvSl0H/xi1+8zTbbbLDBBmuuueaKK66oUcd5aeWc2XRM9/RzHL8S8d0JX+dLDWWZZZZxbgi+EmojyhOkFvr/ysJzykhaGPmyVmfMKbFJG7qoxyuttNK666676aabGg8q7TEUbDr44IMh9c53vvMDH/gACODygx/84Nvf/jaMDj/8cHgde+yxJ598ckh97WtfO+WUUzDu/e9/P3D32WcfNYDsFa94Beh32GEHNTvRxDOe8QzM4u+rrLIKfj360Y+G0bJzytJDcdENXPWJT3ziOuusw5ArDMV1HY6DTvquc6jBzriCeDZSC/+fZV7HDqNgGp0uayQN2gaTfm+++eZs/vKXv5zXoBIucL2DDjqI63E0SvTZz34Wa374wx9+73vf+/jHP97FT33qU8FU+dJQeCWYVELI3vzmN+PmK1/5StAjlxP6BbutttoKBKuttho4AgtfnGDcs571LK4KTedoDibH9dZbz7+Oj3/843U4HJ2jZ8LnSlxbcigTpOYlS2UBSZv3SjBF4GBirplDYQq1a0N72JR+53GNkGzjFEQw6KijjsrXKDoSfeYzn3n3u9/9rne965BDDkEizDrmmGM+97nPgcnx0EMPxUQQAzrBApzz9OtNb3qTf+EldDzpSU8yTniBg5bpABAdX/SiFxGB5z3vec985jMJgjiwxhpruLMjJvqK65tsskloAl04wjvDSewo0pRUeSx5siPihUIQrDgUOu3IXHmyKgDvnoKdom3tcYpddtkFRgYJJgAZGBQiFI7AAqfo94lDcc6/3GnkH/zgB/0LKe4JJrqOcbw1z33rW9+KR/8+FNrnovsJmYaomGgoJmr6ta99LRZrDo577bUXduMU+CCi/8avq84xCCiA0+cttthCpIYXsDbccMOnPe1pxVb+YdQ8bwpcRsEfXYkfGT93jZbQhbF61x6KitRIiYL80UNxzjg+2n777WFk5Lxp3333pb5cwwCEdj1GLsNGImB99atfPfXUU7///e8TddDQIyDCxb/f/e53eSUJQyuJAnzBIfYZgPEYzFOe8hTMfe5zn4u2QqTvipJazDbgcyKfcBuOGK3+rz4UADk2Fh+pxD06Ru+QbquhOAEc33SPtox0ghR2QAQ00QRM/g34eMSN1Q57wButGhmNoOqlGkGgE26GJqPpHO4YFTiIFLUyMB0Szt1ZYkWwjJwkffGLX+R9p59++k9+8hM8MmCg8L6ThwIvKvblL38Zs4g9jhgDS+iJoa46FCOBHaXnyLgJevd//vOfd4U28TVA5FM6ABHnatAl1/WHhBmCfu45FJaAOwUEE0IlZ5pzpFFTuFghACJCZmxaZihLz1MiGhDdzMW23nprbFcvPdpxxx0d6QItoB1gQnsCoU94y3mrH+PYVi+xjFPwI/TBnTPPPPOss84CEKRwgdMZ82mnnYZrGEfgsQ9N+JqhJrcqZEIWYglN8y8aFxm//vWvqwehXASBtnRV5oXp5EwSK2Iytt5uu+22PqViKtcZ3WY/KuHTIA6s+DjhlMEbee5qbK4WLPWmdC6dSqqgyZK6S7ZZCbN408FD0ZLRsj8P0nu3kbNcuEqMMOZrJWaxPLLIquQKiIBQBskNEUp+ACOQIYvrbjY2WkO2GdxI8AtDjd/IWcUNUDBg3PzCF74gYgBO/bIweOkPjQMES1AGBt5lKCQv+osnRRUfMTyTq18rOMhRgDNJ05kdchsNRT8opQ5xY/CtMhRjzvuKCG7mt2AyYHUhrfyI7hqhDMjIGdMAtCESqSSqKgwQrXxdP8R4usv4MnJgxan0/j+HIkugUxyK5LEHy4PDt2gKEVAD94dafDFmI4caRLBShWxm8IVdXwQH7FSuxSoEGSYSAbA6+hamg1v/VSVcSmtBgcg6LO5NwQ4DRtEth+IkUCECl7zUDfpEEQKInMNURWVMGKsHBEJ7gHNFM5DyXSiDhkz4Ll3gLIiAHcaGCBRBp7EJcWDtHOiE2QjBh1Z0ypjJWQYHB6QQhIOAiWn1kCsYjCGpVs+hH1iA5rbw8sVYbxpA+EGmidpyg247isJ0wz2OohBh4ew8Bm9YQv2TCY3BQMHYeKxOjFNZ9gGQ4dE8XdQDKPiaMbMnmHTLlXqAEeKOkYPAGCACTRDH1gQVdi76lDHUTzXYkKJRFl4Gr8OHAjIw0akLh8ITfcSnDA+4GgWKPiNp3s0nCvPVzMY8SLXGr9oysqDRVSSCFLw0pBVoEkpRhSsAzre4p7yPz6oEuYBAzgQHGcLE+9YbikDgDvcBVXaHt30HHPz/wKFIlPQGL/QYfK4fffTRbA4pRoAjonJscgiaUjg+iIbMUloMaAaA1L5DwUdfJ95se+RQDIwwU67zzjvvggsu4IZGCyx+ASOV6DDtI3x0kzsn7U1TVI5ZdNqkR5dOOOEEQDiqQaILIK7XlNP1n//85z/60Y+o4U9/+lM+ASaaRQHQyteFWt1jTsPUW/nTFAbZYigcCgocGDvA5JtOdD26cnJV7DwUOLKPG4zKOFFXjYiWi7USUhQXB0RM9jekchmjAiXggK43RNeQfvzjH0PkiKFwNEYWEM8//3xHDkihtYUyKAmmcXaWkrqi2nWHwiqghBRY9RkiP/vZz9BTakZDEZM+ED4GUDm2llgAi8hquiQW7/QKTGQE7thKtSeKrtNFEKTQBu9zt+6CGUwaYBPVsaqKoFaeXSbtOgLjLecFdHMrWEhWuUaTJhmGPB4LAMfsKXoLMsDiUIZEuekRF8BQXoNiF198scEYpNZdJ8lcoDhTbl28dgKpAgXB9ZHQjtFGxNgIouZf/vKXLAEsbq63GEQBIcgGPjXA448/HlK83uh8C0x4pG8GxUtUqJWJTjnbZyg8WYf4miOJxRRHlEk1GFaN+u06W3EWLMDhgh2ImwdwOtaGESqVkZWgOSlR0Kp7NhiKfugQF2ZVFtZENtfc5ZdfzjvkU0ZCUwCaGSDCyGj7+KHAy3neF4txyp2UhJLyDxBA/Ne//vU555z

In [0]:
Normal_train, Normal_test = Normal_df.randomSplit([0.8,0.2])
Covid_train, Covid_test = Covid_df.randomSplit([0.8,0.2])
Viral_Pneumonia_train, Viral_Pneumonia_test = Viral_Pneumonia_df.randomSplit([0.8,0.2])

In [0]:
train_df = Normal_train.unionAll(Covid_train).unionAll(Viral_Pneumonia_train)
test_df = Normal_test.unionAll(Covid_test).unionAll(Viral_Pneumonia_test)

/databricks/spark/python/pyspark/sql/dataframe.py:1505: DeprecationWarning: Deprecated in 2.0, use union instead.
 warnings.warn("Deprecated in 2.0, use union instead.", DeprecationWarning)

In [0]:
train_df = train_df.repartition(100)
test_df = test_df.repartition(100)

In [0]:
featurizer = DeepImageFeaturizer(inputCol="image", outputCol="features", modelName="InceptionV3")

/local_disk0/tmp/1594595344232-0/PythonShell.py:1: DeprecationWarning: Call to deprecated class DeepImageFeaturizer. (DeepImageFeaturizer will be removed in the next major Databricks Runtime ML release. Please use Pandas UDF instead.)
 from __future__ import absolute_import

In [0]:
lr = LogisticRegression(maxIter=20, regParam=0.05, elasticNetParam=0.3, labelCol="label")

In [0]:
pipeline = Pipeline(stages=[featurizer, lr])

In [0]:
model = pipeline.fit(train_df)

In [0]:
tested_df = model.transform(test_df)
evaluator = MulticlassClassificationEvaluator(metricName="accuracy")
print("Test set accuracy = " + str(evaluator.evaluate(tested_df.select("prediction", "label"))))

Test set accuracy = 0.9193548387096774